# COGS 118B - Final Project: Unsupervised Shoe Recommender System

## Group members

- Jay Buensuceso
- Mabel Szeto
- Joshua Lapidario
- Sialoi Taa

# Abstract 
In this project, we explore the applications of hierarchical clustering when it relates to a recommendation system for shoes. Utilizing a modified Zappos50k dataset <a name="ayuanote"></a>[<sup>[3]</sup>](#ayu) <a name="kgraumannote"></a>[<sup>[4]</sup>](#kgrauman), we attempt to cluster each shoe into its appropriate shoe category as well as retrieve the top 3 most similar shoes in the dataset. Our solution is to create a pipeline involving preprocessing the data, performing dimensionality reduction on it, clustering via an unsupervised algorithm such as hierarchical clustering, and recommending shoes using those created clusters. Our exploration of dimensionality reduction involved individually using PCA, UMAP, and T-SNE and verifying the performance of each of the solutions. For clustering, we utilized hierarchical clustering with initially 20 clusters then increased over time to 72 clusters. Upon closer inspection and determining that our silhouette score and recommendations were not performing as we’d like, we then turned our attention to first: utilizing grayscale images to reduce our dimensionality even further then using a combination of dimensionality reduction through UMAP and T-SNE. The results of these changes led to a silhouette score of 59% and a recommendation system that recommended shoes by shape and color value (relative lightness or darkness of a color).

# Background

At the core, this project explores the many capabilities of machine learning and heirarchical clustering. Fashion styles, more specifically with regards to shoes in this context, are popular ways for people to express themselves in modern society. With AI being the hot topic being discussed as well, it should come to no surprise that it can be applied to the context of fashion. Visual search for specific items of clothing and product recommendation systems have been developed for the market in recent years. There are a plethora of examples of this application in academia as well <a name="zakaryan"></a>[<sup>[1]</sup>](#zakaryannote).

There are a plethora of examples of this application in both industry and academia. In one instance, Modista, a computer vision and machine learning based retail recommendation service, relies on the three features of shape, texture, and color to provide recommendations for similar shoes. In a study by Borras et al. through a UAB Bellaterra research paper “High-Level Clothes Description based on Colour-Texture and Structural Features”, the authors interpreted five clothing combinations in a graphical manner to try to deduce what someone was wearing in an inputted image. Both of these cases yield interesting insight into machine learning for fashion applications, but are fairly simple in nature <a name="khosla"></a>[<sup>[2]</sup>](#khoslanote).

More complex application involve the use of Convolution Neural Networks (CNNs), one such example comes from a Stanford research paper by Khosla and Venkataraman entitled “Building Image-Based Shoe Search Using Convolutional Neural Networks”.[1] CNNs are a popular form of neural networks used in image classification by using the mathematical property of convolution to simplify computations during model training. In this paper, Khosla et. al. struggled in regards to the nature of retrieval being inherently subjective in comparison to the classification (because there is no concrete definition for a shoe to be “similar” to another shoe). In the end, the authors found that they were able to improve over traditional computer vision and machine learning techniques through the use of their deep learning CNN architectures <a name="khosla"></a>[<sup>[2]</sup>](#khoslanote).

# Problem Statement

We made a project to help when you have a pair of shoes and want to get reconmendations on shoes of similar styles. The algorithm will take pictures of shoes as input. We will be utilizing mutiple unsupervised dimensionality reduction techniques to cluster shoes together by their features, exluding color to make the processing less convoluted. After clustering, the algorithm will recomend 3 shoes from the same cluster as the choosen shoe as recomentations. 

# Data

We used data from the UT Zappos50K dataset, a 291MB archive composed of images scrapped from Zappos.com by a research group at the University of Texas. https://vision.cs.utexas.edu/projects/finegrained/utzap50k/ <a name="ayuanote"></a>[<sup>[3]</sup>](#ayu) <a name="kgraumannote"></a>[<sup>[4]</sup>](#kgrauman).  

The dataset had consistent image sizes of 136 by 102 pixels, all images were in color, had clean white backgrounds, and minimal variation in shoe orientation. From the Shoes category we uploaded photos from the Sneakers and Athletic Shoes functional type folder, we did not use all the brands and shoe pictures in the brand because a visual inspection showed that “Athletic Shoes” varied from roller skates, hiking boots, to soccer cleats, whereas we only wanted sneakers for street wear. There were some issues with image compression as some shoes did not keep their original ratio so we avoided using those photos as well, this typically happened by brand so we avoided those brands. We uploaded a total of 2182 sneaker images for our training set, separated into folders by brand. For our test set we uploaded 20 sneaker images also from the UT Zappos50K dataset from differing brands. We chose to not use the metadata as we are making a primarily unsupervised project and the metadata was functionally expired as searching the current Zappos website not all the shoes are being sold today and another item on the website will come up instead. 

# Proposed Solution

Our proposed solution aims to provide recommendations for inputted shoes, based on clusters our unsupervised algorithm creates on existing shoes within our model. The algorithm will take pictures of shoes as input, which will be preprocessed into arrays of numeric pixel values. We will be utilizing dimensionality reduction techniques we learned in class (such as PCA, t-SNE, and UMAP) to give our clustering algorithm the chance to cluster shoes more efficiently and with less variability, excluding color through gray-scaling to make the feature processing less convoluted. The clustering algorithm of our choice is Agglomerative/Hierarchical Clustering, since we could focus less on finding the specific number of clusters for our model (since agglomerative clustering essentially handles it for us) and instead focus on the quality of those clusters. After clustering, the algorithm will recommend 3 shoes from the same cluster to gauge what kind of shoes could be similar to the inputted shoe, taking into account the similarities and patterns learned from our clustering.


# Evaluation Metrics

We used silhouette score as our primary measurement since it measures how similar an object is to its own cluster compared to other clusters. Additionally, our data is not labeled, while we have images separated by brand, some brands have hundreds of shoes of varying styles within the folder,  and thus other methods like Adjusted Rand Index, Normalized Mutual Information, and F1 Scores are out of the question. In addition, in the examples below in our results section for our final model, you can see a sample of shoes within the cluster for a visual inspection. As you can see the shoes separate out by different features such as shoe shape, color, and even style. This of course is ultimately a subjective way of interpreting the reliability of our recommender and could vary from person-to-person.

# Results

## Preprocessing
For preprocessing we first find the mean of the dataset. After finding the mean, it is subtracted from the dataset to mean center the data. Then we initialize UMAP and TSNE objects to begin dimensionality reduction. The mean centered data is using the UMAP object to go from a 13872 dimensional space to a 100 dimensional one. After the UMAP dimensionality reduction, the TSNE object is used to go from a 100 dimensional space to a 2 dimensional one. 

```python
# Preprocess the data
# Because of the curse of dimensionality, we'll use UMAP to reduce the dimensionality
image_collection_mean = np.mean(image_collection, axis=0, keepdims=True)

# Mean centered data
mean_centered_data = image_collection - image_collection_mean

n_components = 2
# Best UMAP Params: 10 neighbors, 100 components, 0.05 min_dist
umap_v1 = UMAP(n_neighbors=10, n_components=n_components*50, random_state=99, min_dist=0.05) #lowered the n_neighbors from 40 and the min_dist from 0.07
# Best TSNE Params: 2 components, 15 perplexity
tsne_v1 = TSNE(n_components=n_components,perplexity=15,random_state=99)
reduced_image_collection = umap_v1.fit_transform(mean_centered_data)
reduced_image_collection = tsne_v1.fit_transform(reduced_image_collection)
```
As seen in: [Final Recommender](Mixture.ipynb)

## Agglomerative/Hierarchical Clustering
We first explored our data by viewing 4 linkage trees via hierarchical clustering and determining the number of clusters based on inspection of that result. 

![image](imgs/Dendrogram.png)
*Graph from our notebook: [UMAP Implementation w/ Agglomerative Clustering](UMAP_IMPLEMENTATION_AGGO.ipynb)*

After toying with the parameters, we decided to focus our attention on ward linkage since it’s dendrogram appeared to be the most reliable in terms of producing separable, unique clusters. In the initial stages of our dimensionality reduction with PCA, we tested out by reducing to 200 and then 80 principal components. The dendrograms can been seen below in the PCA section. Unlike some other clustering methods (like k-means for example), agglomerative clustering does not require specifying the number of clusters beforehand. This can be advantageous when we’re unsure about the optimal number of clusters for our shoe image dataset. We were able to worry less about finding the right number of clusters and instead focus our attention on the quality of the clusters we produced via the silhouette score. Thus, we continued to rely on the ward linkage throughout the rest of our project implementation.

## Dimensionality Reduction Exploration
There were 3 dimensionality reduction techniques we explored: PCA, T-SNE, and UMAP.

### PCA
Our first attempt at dimensionality reduction was to use PCA. After performing and fitting our reduction, we obtained the eigenvalues. These represent the amount of variance explained by each principal component. We then obtained the cumulative explained variance for each PC, also known as the sum of explained variances for that component. For further analysis, we were able to plot the explained variance against the number of principal components. This is called a scree plot, and by examining it we were able to see where the explained variance started to level off. 

Visually, this is the point in the graph that represents an elbow. After this point, adding more principal components provides diminishing returns when trying to explain additional variance within the data. Ideally this is the point that will give us the optimal number of principal components to use.
At 80 principal components, it can explain over 80% of the variance in the dataset. At 200 principal components, it only explains 91% of the variance. Thus, we decided on staying at 80 principal components due to the diminishing returns.

However, we noticed there was little variety in the recommendations of our shoes. Namely, we suspected that PCA could not map the non-linearity of our dataset. Thus, we moved our focus to the more intricate dimensionality reduction techniques we learned in class.

"Elbow Method":

![image](imgs/eigenvalues_from_pca.png)

*Graph from our notebook: [PCA Implementation w/ Heirarchical Clustering](Hierarchical_Sneaker_Clustering.ipynb)*

Results from Heirarchical Clustering of 80 Principal Components vs 200 Principal Components:

![image](imgs/80_principal_components.png)
*Graphs from our notebook: [PCA Implementation w/ Heirarchical Clustering](Hierarchical_Sneaker_Clustering.ipynb)*
![image](imgs/200_principal_components.png)

*Graphs from our notebook: [PCA Implementation w/ Heirarchical Clustering](Hierarchical_Sneaker_Clustering.ipynb)*

### UMAP
One of the key components of this project involved preserving the spatial representation from a higher dimension to a lower dimension. During our exploration of PCA, we found that we were not reaching this goal in dimensionality reduction and thus, our next step was refactoring portions of our code to utilize UMAP prior to hierarchical clustering. Additionally, This was also the step where we switched from a Scipy based hierarchical clustering implementation to scikitlearn’s implementation of agglomerative clustering.

For our initial exploration we utilized similar hyperparameter to the UMAP implementation in A4:
    <li>40 nearest neighbors</li>
    <li>2 components</li>
    <li>Random state of 99</li>
    <li>0.05 minimum distance</li>
    
With a cluster amount of 72, our clusters looked as such:

![image](imgs/umap_clusters.png)
*Graph from our notebook: [UMAP Implementation w/ Agglomerative Clustering](UMAP_IMPLEMENTATION_AGGO.ipynb)*

This approach led to a better recommendation of shoes compared to PCA; however, through our visual inspection of multiple test cases, we found that many outliers existed in our clusters. An example of this case was that predominantly white converse and vans existed within a cluster of white athletic running sneakers. When we realized that color was potentially having a greater effect on our clusters, our team split into 2 paths: half our group worked on further optimizing UMAP with grayscale images while the other half started an exploration of dimensionality reduction through T-SNE. 

With that change to our implementation and slightly better recommendations than before, our silhouette score became 0.36410898.

![image](imgs/umap_silhouette.png)
*Graph from our notebook: [UMAP Implementation w/ Agglomerative Clustering](UMAP_IMPLEMENTATION_AGGO.ipynb)*

Our UMAP implementation can be found in this notebook: [UMAP Implementation w/ Agglomerative Clustering](UMAP_IMPLEMENTATION_AGGO.ipynb)

### TSNE
Our T-SNE implementation was almost identical to UMAP’s in terms of implementation, the only differences being in the functions and declarations used in SciKit’s T-SNE library.

![image](imgs/tsne_clusters.png)
*Graph from our notebook: [T-SNE Implementation w/ Heirarchical Clustering](TSNE_IMPLEMENTATION_AGGO.ipynb)*

As for the results, T-SNE proved far inferior to UMAP. The clusters were rather compact, with many possibilities for overlap. This was further evidenced by the poor performance in the silhouette score of 0.019396267657152258 and the silhouette plot below. 

![image](imgs/tsne_silhouette.png)
*Graph from our notebook: [T-SNE Implementation w/ Heirarchical Clustering](TSNE_IMPLEMENTATION_AGGO.ipynb)*


We believe that with the nature of T-SNE to preserve local structure more than global structure, the topology of the shoes from a global perspective was heavily ignored in favor of local comparisons of neighboring shoes to one another. We could not trust that T-SNE accurately reflected the overall distances and relationships between clusters or groups of points in the original space. Based on these results, we figured to be creative about how to approach our dimensionality reduction and find a balance between preserving local and global structure from our original dimensions.

Our T-SNE implementation can be found in this notebook: [TSNE Implementation w/ Agglomerative Clustering](TSNE_IMPLEMENTATION_AGGO.ipynb)

### UMAP + TSNE Combination (Final Model)

We noticed that while UMAP gave us a higher silhouette score, it wasn’t as high as we wanted. We then had the idea of combining UMAP and TSNE in our preprocessing. After discussion and much thought, we came to understand that using UMAP to reduce dimensionality and then use TSNE to further reduce dimensionality would give us better clustering representation.

The benefit of UMAP is that it’s SGD, computational efficiency, and topology preservation from the higher dimension down into the lower dimensions. The benefit of TSNE was to preserve local spatiality relationships in the dataset. The thought to combine these 2 methods of data processing for our preprocessing came from the desire of wanting tighter clustering but increasing the distance between clusters to prevent overlap. The idea was that UMAP would reduce the dimensions of the dataset down to close to the final dimension reduction size (2 dimensions). UMAP, in our model, reduced the dimensions from 13872 dimensions down to 100 dimensions. This is done all while preserving the topology of the dataset from its higher dimension state into its lower dimensional state. 

Why didn’t we reduce it to 2 dimensions from the start? This is because we noticed that we couldn’t just project the data onto a smaller dimensional space and continue clustering. We needed to manipulate the data more into something more distinguishable. This is why we then use TSNE to preserve the spatial locality of a 100 dimensional dataset. 

But if we wanted to preserve spatial locality, why didn’t we use TSNE from the start? This is because the higher dimensional topology still had information we could exploit and extract from. But the problem with TSNE is that it’s computationally inefficient and costly, all the while it doesn’t preserve global topology as well as UMAP. Which is why we used UMAP to reduce the dimensionality of the dataset while preserving the topology so we can get a dataset that could have as close of a complete translation and representation of the dataset from the higher dimension to the lower dimension. Now TSNE’s computation cost is drastically reduced and we can focus on the local spatial information surrounding individual points and its neighbors. This is the part we want to heavily focus on and extract information from. Because if we focus on data points and its nearest neighbors, we can create more compact and accurate clustering, with minimal computation costs, AND keep the distance between clusters relatively the same as they would be in high dimensional spaces. Data points in high dimensional spaces can be extremely far away from each other due to the number of dimensions, which is what we want. We want to exploit the distance between points in high space and bring that into a lower space. Then we look at each point’s nearest neighbors and keep that relative distance the same when moving even lower in dimensions. This is why our method works. We exploit the characteristics and flaws of high dimensional datasets and bring them down to lower dimensions to be used as a way to increase the accuracy of our clustering. The points far from each other in topology will move away from each other when moving to lower dimensions and points closer to each other will do the opposite. After using TSNE, we create compact and focused clusters.

Here is a UMAP clustering:

![image](imgs/umap_clusters.png)
*Graph from our notebook: [UMAP Implementation w/ Heirarchical Clustering](UMAP_IMPLEMENTATION_AGGO.ipynb)*


Then here is the UMAP + TSNE clustering:

![image](imgs/umap_tsne_cluster_2.png)
*Graph from our notebook: [Final Recommender](Mixture.ipynb)*

The UMAP + TSNE clustering shows very clear and compact clusters, while the UMAP clustering has tight clustering but has too much of a chance of overlapping with other clusters. This overlapping causes incorrect clustering of points and lowers the silhouette score. Our UMAP + TSNE clustering notebook can be found here: [Final Recommender](Mixture.ipynb)

# Discussion

### Interpreting the result

Here is the clustering used with our final proposed solution of using UMAP + TSNE for our preprocessing and hierarchical clustering to learn our clusters:

![image](imgs/umap_tsne_cluster_2.png)
*Graph from our notebook: [Final Recommender](Mixture.ipynb)*

These are very compact and distinguishable clusters, which shows that the clustering found patterns in similar points of data. These patterns were enough to cluster into 72 different clusters! The choosing of the number of clusters was done by trial and error with the silhouette score as our guide.

Here’s what we see is inside cluster 10:

![image](imgs/cluster_10.png)
*Visualizations from our notebook: [Final Recommender](Mixture.ipynb)*

It looks like cluster 10 is clustering low cut white colored shoes. This is an example of a cluster that found a pattern in the color and shape of the shoe. Since we included a grayscale function in processing the data, very light colored shoes and black shoes clustering together makes sense because their color value was already at one extreme of the spectrum so the grayscale had little effect on their clustering. For other cases where the shoes might be all one color like red or have multicolor patterns, the clustering struggled much more staying within a cohesive color palette but the overall color value or intensity were relatively similar. 

Here’s what we see in cluster 25:

![image](imgs/cluster_25.png)
*Visualizations from our notebook: [Final Recommender](Mixture.ipynb)*

This time it’s low cut dark colored shoes. This notebook will be available for anyone to use to see how the clustering works.

Here’s the test shoe recommender system. If the clusterings above are accurate, then if we place a test image into our model it should be able to predict what cluster it would belong to and recommend shoes based off of that input.
Below are some test shoes that received some recommendations after being placed through the model:

![image](imgs/rec_shoes.png)
*Visualizations from our notebook: [Final Recommender](Mixture.ipynb)*

The shoes used for input are on the far left and the recommended shoes are the 3 after it. Visually it can be seen that the shoes inside the dataset are accurately clustered after finding a pattern to base the cluster off of. The first shoe was placed in a cluster that had dark colored running shoes. The second shoe was placed in a cluster that had light colored shoes with white laces and stripes on the side of soles. The third shoe was placed in shoes that were darker in color and had different textures (leather and suede)  than others and had the same slender shape as the input shoe. These recommendations show accurate clusterings and represent how well the clusters successfully compacted together similar data points.

Finally is the silhouette score. 

Here’s the TSNE silhouette score:

![image](imgs/tsne_silhouette.png)
*Graph from our notebook: [T-SNE Implementation w/ Heirarchical Clustering](TSNE_IMPLEMENTATION_AGGO.ipynb)*

__Silhouette Score: -0.019396267657152258__

This is a terrible score and it indicates that we’re using the wrong preprocessing method or the wrong clustering algorithm or even both! Values near 0 mean we have overlapping clusters. Negative values generally mean that a sample has been assigned to the wrong cluster, as a different cluster is more similar to it than the current cluster.

Then the UMAP silhouette score:

![image](imgs/umap_silhouette.png)
*Graph from our notebook: [UMAP Implementation w/ Heirarchical Clustering](UMAP_IMPLEMENTATION_AGGO.ipynb)*

__Silhouette Score: 0.36410898__

This is much better but not enough to say that this model for clustering is great or is reliable to cluster data together with a specific pattern. The score is over 0.25 but under 0.5 so that means it still is weak. 

![image](imgs/mixture_silhouette.png)
*Graph from our notebook: [Final Recommender](Mixture.ipynb)*

__Silhouette Score: 0.59216785__

This is a great score as 0.7 is considered to be strong, and a value over 0.5 reasonable. This plot and score is saying that most of the data points clustered have a high likelihood of being in the correct cluster while there is an extremely small portion of the dataset that believes it might belong to the wrong cluster. Almost reaching a silhouette score of 0.6 demonstrates the quality of the model and the clustering techniques used to create it. 
The result of our clustering system placed on the Zappos50k dataset is that it reliably clusters shoes of either similar shape, color value, or both and is able to distinguish between each cluster with high confidence.

### Limitations

For the Zappos50K dataset, there is a limitation for the shoe recommendation as the dataset images have product numbers from a fixed point in time, to do the recommendations properly would rely on the image names to be update with the current site's CID system (the system used for the product's unique ID on Zappos) to get the shoe name, brand, and price. 
We did explore the effects of changing different hyperparameters and realized when we got our silhouette score to 0.69, the recommendation got worse. This is likely due to overfitting and is visble when looking at the sample visualizations from random clusters.   

### Ethics & Privacy

As with any computer vision or machine learning problem, we need to be considerate with the consent and privacy of any individuals appearing within or affiliated with the dataset. We ensured that the data we used for our project did not infringe upon anyone’s privacy or involved personal information. We did not make a generative AI that creates new images from the images in our datasets so we are not infringing on copyright for the images and are purely using the images for an academic project which we do not gain a profit from.

A bias arises from an uneven distribution of categorized shoes that we took as our input, which may result in certain shoes and shoe styles having inaccurate predictions or irrelevant recommendations. This of course, would make our model less reliable and discourage people with diverse styles from using it. We were not able to include sneakers from all brands and even the brands we did upload photos from, the amount of images is not evenly distributed and not up to the latest trends since this data was from 2016 and from one website only. Because of our limited time to execute on this project we lessened the scope of our project down to a specific style of shoe (sneakers) to test the validity of our model.

By making our methods and analysis public, we open our project to those that can point out approaches that we have not considered and so it can be reproduced and expanded upon. We provide sample visualizations from our final clusters as we do not have a ground truth to compare against to allow people to visually inspect for themselves and draw their own interpretations from our results. The algorithms we used were all unsupervised which means it may have picked up biases from the dataset that we tried to mitigate through parameter tuning.   


### Conclusion

We wanted to create a shoe recommendation system that works purely on images given with no other labeling information. We were able to achieve this with our implementation of UMAP and t-SNE on the Zappos50K dataset, resulting in clusters of similar styled shoes categorized by shape/style as seen in the samples of the cluster images. Our approach aligns with contemporary functionalities like reverse image search employed by industry giants such as Google and Pinterest, as well as recommender systems prevalent on major retail platforms like Amazon, Macy’s, and Nordstrom to name a few. While these platforms typically rely on deep learning algorithms and customer metadata for recommendations, our unsupervised approach focusing solely on image content showcases a simpler, yet still informative alternative methodology. Future work would be increasing the ways we can validate our model and testing it with images of shoes that are not all oriented in the same way with the same background, and make the algorithm to be able to distinguish by color. Overall, we were very satisfied with the results of our unsupervised shoe recommender, and would even use it ourselves to find shoes that fit our own fashion preferences!

# Footnotes
<a name="zakaryannote"></a>1.[^](#zakaryan):  Zakaryan, Vahan. “AI Clothing Detection: Use Cases for Fashion and E-Commerce.” *Postindustria*, 17 June 2022, https://postindustria.com/ai-clothing-detection-use-cases-for-fashion-and-e-commerce/
<br>

<a name="khoslanote"></a>2.[^](#khosla): Khosla, Neal and Venkataraman Vignesh. “Building Image-Based Shoe Search Using Convolutional Neural Networks" *Cs231n.Stanford.Edu,* 2015, http://cs231n.stanford.edu/reports/2015/pdfs/nealk_final_report.pdf
<br> 

<a name="ayuanote"></a>3.[^](#ayu): A. Yu and K. Grauman. "Fine-Grained Visual Comparisons with Local Learning". In CVPR, 2014, https://vision.cs.utexas.edu/projects/finegrained/utzap50k/
<br> 

<a name="kgraumannote"></a>4.[^](#kgrauman): A. Yu and K. Grauman. "Semantic Jitter: Dense Supervision for Visual Comparisons via Synthetic Images". In ICCV, 2017, https://vision.cs.utexas.edu/projects/finegrained/utzap50k/ 